In [0]:
# ------------------------------------------------------------
# Databricks Widgets
# Used to parameterize the notebook for Jobs / reusability
# ------------------------------------------------------------
dbutils.widgets.text("s3_source_path", "", "S3 Source URI (CSV)")
dbutils.widgets.text("s3_target_path", "", "S3 Target URI (Parquet)")
dbutils.widgets.text("start_year", "", "Start Year")
dbutils.widgets.text("end_year", "", "End Year")
dbutils.widgets.text("table_name", "", "Target Table Name")
dbutils.widgets.text("catalog_name", "movielens", "Catalog Name")
dbutils.widgets.text("schema_name", "bronze", "Schema Name")

# ------------------------------------------------------------
# Read widget values
# ------------------------------------------------------------
s3_source_path = dbutils.widgets.get("s3_source_path")
s3_target_path = dbutils.widgets.get("s3_target_path")
start_year = dbutils.widgets.get("start_year")
end_year = dbutils.widgets.get("end_year")
table_name = dbutils.widgets.get("table_name")
catalog_name = dbutils.widgets.get("catalog_name")
schema_name = dbutils.widgets.get("schema_name")

In [0]:
# ------------------------------------------------------------
# Input validation
# Fail fast for misconfigured jobs
# ------------------------------------------------------------
if not s3_source_path or not s3_source_path.startswith("s3://"):
    raise ValueError(f"CONFIGURATION ERROR: Invalid source path '{s3_source_path}'")

if not s3_target_path or not s3_target_path.startswith("s3://"):
    raise ValueError(f"CONFIGURATION ERROR: Invalid target path '{s3_target_path}'")

if not start_year:
    raise ValueError("CONFIGURATION ERROR: start_year not passed")

if not end_year:
    raise ValueError("CONFIGURATION ERROR: end_year not passed")

if not table_name:
    raise ValueError("CONFIGURATION ERROR: table_name not passed")

# Normalize S3 paths to avoid malformed file paths
if not s3_source_path.endswith("/"):
    s3_source_path += "/"

if not s3_target_path.endswith("/"):
    s3_target_path += "/"

print("[INFO] Validation passed")

In [0]:
# ------------------------------------------------------------
# Explicit schemas for Bronze ingestion
# Schema inference is intentionally avoided
# ------------------------------------------------------------
from pyspark.sql.types import (
    StructType, StructField, IntegerType, DoubleType, LongType, StringType
)

ratings_schema = StructType([
    StructField("userId", IntegerType(), True),
    StructField("movieId", IntegerType(), True),
    StructField("rating", DoubleType(), True),
    StructField("timestamp", LongType(), True)
])

tags_schema = StructType([
    StructField("userId", IntegerType(), True),
    StructField("movieId", IntegerType(), True),
    StructField("tag", StringType(), True),
    StructField("timestamp", LongType(), True)
])

schemas = {
    "ratings": ratings_schema,
    "tags": tags_schema
}

# Validate supported table names
if table_name not in schemas:
    raise ValueError(
        f"Unsupported table_name '{table_name}'. "
        f"Supported: {list(schemas.keys())}"
    )

# Fully qualified Unity Catalog table name
full_table_name = f"{catalog_name}.{schema_name}.{table_name}"

print(f"[START] Job Initialized for table {full_table_name}")
print(f"Source: {s3_source_path}")
print(f"Target: {s3_target_path}")

In [0]:
# ------------------------------------------------------------
# Historical ingestion logic
# Iterates year-wise and appends data to Bronze Delta table
# ------------------------------------------------------------
def process_historical_data():
    total_files = 0

    for year in range(int(start_year), int(end_year) + 1):
        source_file = f"{s3_source_path}{table_name}_{year}.csv"
        print(f"Processing Year: {year} | Source: {source_file}")

        try:
            df = (
                spark.read
                    .format("csv")
                    .option("header", "true")
                    .schema(schemas[table_name])
                    .load(source_file)
            )

            (
                df.write
                    .format("delta")
                    .mode("append")  # Historical loads are appended year by year
                    .save(s3_target_path)
            )

            total_files += 1
            print(f"Appended data for year {year}")

        except Exception as e:
            # Missing or corrupt yearly files are skipped intentionally
            print(f"Skipping year {year}: {e}")

    return total_files

In [0]:
# ------------------------------------------------------------
# Execute ingestion
# ------------------------------------------------------------
files_processed = process_historical_data()

expected_files = int(end_year) - int(start_year) + 1
if files_processed < expected_files:
    print(
        f"WARNING: Expected {expected_files} files, "
        f"but processed only {files_processed}"
    )

# ------------------------------------------------------------
# Register Delta table in Unity Catalog
# Storage and metadata are intentionally decoupled
# ------------------------------------------------------------
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {full_table_name}
    USING DELTA
    LOCATION '{s3_target_path}'
""")

print("[END] Job completed")